In [161]:
import pandas as pd
import numpy as np
import os

In [162]:
os.chdir('/Users/chrissoria/Documents/Research/BICS_Political_Polarization/')

In [163]:
col_types = {
    'FIPS_Code': str,
    'District': str,
    'CD':str
}

CD_df = pd.read_csv('data/House_Reps_District.csv', dtype=col_types)

some ZIP Codes, such as "unique" ZIPs that are often tied to an internal mail-routing mechanism (such as UC Berkeley which has its own internal ZIP code and mail-routing system) will have very low ratios). \
As of 12/23 I'm realzing that I'll need to start with a complete list of all ZIP codes (from USPS) before I start this so I can get a good sense of completeness. 

Below, I'm reading in that full list of ZIP codes

In [164]:
ZIP_to_CD_df = pd.read_excel('Data/ZIP_CD_062020.xlsx')

ZIP_to_CD_df = ZIP_to_CD_df.rename(columns={
    'RES_RATIO': 'CD_RES_RATIO',
    'BUS_RATIO': 'CD_BUS_RATIO',
    'OTH_RATIO': 'CD_OTH_RATIO',
    'TOT_RATIO': 'CD_TOT_RATIO'
})

ZIP_to_CD_df['ZIP'] = ZIP_to_CD_df['ZIP'].astype(str).str.zfill(5)

In [165]:
col_types = {
    'DELIVERY ZIPCODE': str
}

FULL_ZIPS = pd.read_excel('Data/Full_Zips.xlsx', dtype=col_types)

print(len(FULL_ZIPS))

FULL_ZIPS = FULL_ZIPS[['DELIVERY ZIPCODE','PHYSICAL STATE','TYPE']]

FULL_ZIPS = FULL_ZIPS.rename(columns={
    'DELIVERY ZIPCODE': 'ZIP',
    'PHYSICAL STATE': 'USPS_STATE'
})

#removing ZIP codes that aren't in US main
FULL_ZIPS = FULL_ZIPS.drop_duplicates(subset='ZIP', keep='first')
FULL_ZIPS = FULL_ZIPS[pd.notna(FULL_ZIPS['USPS_STATE'])].reset_index(drop=True)
exclude_states = ["VI", "PR", "MH", "MP", "FM", "GU"]
FULL_ZIPS = FULL_ZIPS[~FULL_ZIPS['USPS_STATE'].isin(exclude_states)].reset_index(drop=True)
FULL_ZIPS = FULL_ZIPS[~FULL_ZIPS['TYPE'].isin(["Other", "Unique"])].reset_index(drop=True)

print(len(FULL_ZIPS))
FULL_ZIPS.head()

49649
37714


/Users/chrissoria/anaconda3/lib/python3.11/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


,ZIP,USPS_STATE,TYPE
0,01001,MA,Standard/PO BOX
1,01002,MA,Standard/PO BOX
2,01004,MA,Standard/PO BOX
3,01005,MA,Standard/PO BOX
4,01007,MA,Standard/PO BOX


After merging, I find that many of the Alaska ZIP codes aren't getting matched. Since there's only one CD in Alaska, I'll fill in any Alaska state ZIP with 0200. \
This data will not match PO boxes. Most non matches are coming from PO boxes.

In [166]:
ZIP_to_CD_df = pd.merge(FULL_ZIPS, ZIP_to_CD_df, on = "ZIP", how = "left")

ZIP_to_CD_df['CD'] = ZIP_to_CD_df.apply(lambda row: "0200" if row['USPS_STATE'] == "AK" else row['CD'], axis = 1)
ZIP_to_CD_df['CD_TOT_RATIO'] = ZIP_to_CD_df.apply(lambda row: 1.0 if row['USPS_STATE'] == "AK" else row['CD_TOT_RATIO'], axis = 1)

ZIP_to_CD_df

,ZIP,USPS_STATE,TYPE,CD,CD_RES_RATIO,CD_BUS_RATIO,CD_OTH_RATIO,CD_TOT_RATIO
0,01001,MA,Standard/PO BOX,2501,1.000000,1.0,1.0,1.000000
1,01002,MA,Standard/PO BOX,2502,1.000000,1.0,1.0,1.000000
2,01004,MA,Standard/PO BOX,2502,1.000000,1.0,1.0,1.000000
3,01005,MA,Standard/PO BOX,2502,1.000000,1.0,1.0,1.000000
4,01007,MA,Standard/PO BOX,2502,0.998118,1.0,1.0,0.998217
...,...,...,...,...,...,...,...,...
44276,99925,AK,Standard/PO BOX,0200,0.000000,0.0,1.0,1.000000
44277,99926,AK,Standard/PO BOX,0200,0.000000,0.0,1.0,1.000000
44278,99927,AK,Standard/PO BOX,0200,0.000000,0.0,1.0,1.000000
44279,99928,AK,Standard/PO BOX,0200,0.000000,0.0,1.0,1.000000


For now, I will assign a ZIP code to whatever whatever County it falls mostly under residentially \
This will NOT include DC simply because DC doesn't have a CD \
IN A FUTURE VERSION, I WILL HAVE TO START WITH A LIST OF ZIP CODES THAT INCLUDE DISTRICT OF COLUMBIA'S (WHICH I  HAVE ABOVE)

In [167]:
col_types = {
    'COUNTY': str,
    'ZIP': str
}

ZIP_to_County_df = pd.read_excel('Data/ZIP_COUNTY_062020_HUD.xlsx', dtype=col_types)
ZIP_to_County_df['COUNTY'] = ZIP_to_County_df['COUNTY'].astype(str).str.zfill(5)
ZIP_to_County_df['ZIP'] = ZIP_to_County_df['ZIP'].astype(str).str.zfill(5)

ZIP_to_County_df = pd.merge(ZIP_to_CD_df, ZIP_to_County_df, how ='outer', on = "ZIP")

# Sort the DataFrame by RES_RATIO in descending order
ZIP_to_County_df = ZIP_to_County_df.sort_values(by='RES_RATIO', ascending=False)
ZIP_to_County_df = ZIP_to_County_df.drop_duplicates(subset='ZIP', keep='first')

ZIP_to_County_df.reset_index(drop=True, inplace=True)

ZIP_to_County_df = ZIP_to_County_df.rename(columns={
    'RES_RATIO': 'COUNTY_RES_RATIO',
    'BUS_RATIO': 'COUNTY_BUS_RATIO',
    'OTH_RATIO': 'COUNTY_OTH_RATIO',
    'TOT_RATIO': 'COUNTY_TOT_RATIO'
})

ZIP_FEATURES = pd.merge(ZIP_to_CD_df, ZIP_to_County_df)

I'm realizing I'll have to merge the school district data onto ZIP_to_County_df (give that there's some zip codes that don't have a school district associated) \
Yet, the issue of some ZIP codes falling through the cracks will still be there. \
I'll need something that goes from county and matches to all ZIP codes within OR \
Something that matches ZIP codes directly to the nearest school district (IDEAL). \
For now (12/26/23) this variable is not complete. 

Instead, I'll match the ZIP code to a census tract THEN match tract onto school district

In [168]:
col_types = {
    'TRACT': str,
    'ZIP': str
}

ZIP_to_Tract = pd.read_excel('data/ZIP_TRACT_062020.xlsx', dtype=col_types)

ZIP_to_Tract = ZIP_to_Tract.sort_values(by='RES_RATIO', ascending=False)
ZIP_to_Tract = ZIP_to_Tract.drop_duplicates(subset='ZIP', keep='first')

ZIP_to_Tract.reset_index(drop=True, inplace=True)

ZIP_FEATURES = pd.merge(ZIP_FEATURES, ZIP_to_Tract, on = "ZIP", how = "left")

ZIP_FEATURES

,ZIP,USPS_STATE,TYPE,CD,CD_RES_RATIO,CD_BUS_RATIO,CD_OTH_RATIO,CD_TOT_RATIO,COUNTY,COUNTY_RES_RATIO,COUNTY_BUS_RATIO,COUNTY_OTH_RATIO,COUNTY_TOT_RATIO,TRACT,RES_RATIO,BUS_RATIO,OTH_RATIO,TOT_RATIO
0,01001,MA,Standard/PO BOX,2501,1.000000,1.0,1.0,1.000000,25013,1.000000,1.000000,1.000000,1.000000,25013813209,0.404431,0.442809,0.250000,0.403828
1,01002,MA,Standard/PO BOX,2502,1.000000,1.0,1.0,1.000000,25015,0.988689,0.998889,0.997312,0.989909,25015820500,0.280456,0.602222,0.548387,0.318705
2,01004,MA,Standard/PO BOX,2502,1.000000,1.0,1.0,1.000000,25015,1.000000,1.000000,1.000000,1.000000,25015820500,1.000000,1.000000,1.000000,1.000000
3,01005,MA,Standard/PO BOX,2502,1.000000,1.0,1.0,1.000000,25027,1.000000,1.000000,1.000000,1.000000,25027722100,0.992387,1.000000,0.985401,0.992145
4,01007,MA,Standard/PO BOX,2502,0.998118,1.0,1.0,0.998217,25015,1.000000,1.000000,1.000000,1.000000,25015820203,0.591843,0.619883,0.800000,0.598277
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37709,99925,AK,Standard/PO BOX,0200,0.000000,0.0,1.0,1.000000,02198,0.000000,0.000000,1.000000,1.000000,02198000200,0.000000,0.000000,1.000000,1.000000
37710,99926,AK,Standard/PO BOX,0200,0.000000,0.0,1.0,1.000000,02198,0.000000,0.000000,1.000000,1.000000,02198940100,0.000000,0.000000,1.000000,1.000000
37711,99927,AK,Standard/PO BOX,0200,0.000000,0.0,1.0,1.000000,02198,0.000000,0.000000,1.000000,1.000000,02198000100,0.000000,0.000000,1.000000,1.000000
37712,99928,AK,Standard/PO BOX,0200,0.000000,0.0,1.0,1.000000,02130,0.000000,0.000000,1.000000,1.000000,02130000100,0.000000,0.000000,1.000000,1.000000


Now, let's match this tract to a school district. \
After merging this file, I find that there are still a large chunk of rows without a school district \
For these rows, I'll use a cruder approximation of school district at the county-level (which might be too broad)

In [169]:
col_types = {
    'LEAID': str,
    'TRACT': str
}

school_to_tract = pd.read_excel('data/grf20_lea_tract.xlsx', dtype=col_types)


ZIP_FEATURES = pd.merge(ZIP_FEATURES, school_to_tract, on = "TRACT", how = "left")

Below, I'll read in the county-district level data

In [170]:
col_types = {
    'LEAID': str,
    'STCOUNTY': str
}

County_to_District = pd.read_excel('data/grf20_lea_county.xlsx', dtype=col_types)
County_to_District = County_to_District.sort_values(by='LANDAREA', ascending=False)
County_to_District = County_to_District.drop_duplicates(subset='STCOUNTY', keep='first')

County_to_District = County_to_District.rename(columns={"STCOUNTY": "COUNTY"})

County_to_District = County_to_District[['NAME_LEA20','COUNTY','LEAID']]

ZIP_FEATURES = pd.merge(ZIP_FEATURES, County_to_District, on = "COUNTY", how = "left")

Next, I'll clean up the data so that we keep only the desired columns, differentiate between the census tract (higher resolution) school district match and the county-level (lower resolution) match. Lastly, I'll create one column for school district based on both of these types of matches

In [171]:
ZIP_FEATURES.head()

columns_to_keep = ['ZIP','USPS_STATE','CD','COUNTY','TRACT','LEAID_x','LEAID_y','NAME_LEA20_x','NAME_LEA20_y']

ZIP_FEATURES = ZIP_FEATURES[columns_to_keep]

ZIP_FEATURES = ZIP_FEATURES.rename(columns = {
                                   'NAME_LEA20_x': 'TRACT_SCH_DISTRICT',
                                   'NAME_LEA20_y': 'COUNTY_SCH_DISTRICT',
                                   'LEAID_x': 'TRACT_LEAID',
                                   'LEAID_y': 'COUNTY_LEAID'})

ZIP_FEATURES['SCHOOL_DISTRICT'] = ZIP_FEATURES.apply(lambda row: row['COUNTY_SCH_DISTRICT'] if pd.isna(row['TRACT_SCH_DISTRICT']) else row['TRACT_SCH_DISTRICT'], axis = 1)
ZIP_FEATURES['LEAID'] = ZIP_FEATURES.apply(lambda row: row['COUNTY_LEAID'] if pd.isna(row['TRACT_LEAID']) else row['TRACT_LEAID'], axis = 1)


ZIP_FEATURES.head()

,ZIP,USPS_STATE,CD,COUNTY,TRACT,TRACT_LEAID,COUNTY_LEAID,TRACT_SCH_DISTRICT,COUNTY_SCH_DISTRICT,SCHOOL_DISTRICT,LEAID
0,01001,MA,2501,25013,25013813209,2501800,2505160,Agawam School District,Gateway School District,Agawam School District,2501800
1,01002,MA,2502,25015,25015820500,2501890,2505740,Amherst School District,Hampshire School District,Amherst School District,2501890
2,01002,MA,2502,25015,25015820500,2501920,2505740,Amherst-Pelham School District,Hampshire School District,Amherst-Pelham School District,2501920
3,01004,MA,2502,25015,25015820500,2501890,2505740,Amherst School District,Hampshire School District,Amherst School District,2501890
4,01004,MA,2502,25015,25015820500,2501920,2505740,Amherst-Pelham School District,Hampshire School District,Amherst-Pelham School District,2501920


In [172]:
schools_closures = pd.read_csv('data/District_Monthly_Shares_03.08.23.csv',dtype=str)

columns_to_keep = ['NCESDistrictID','month','share_inperson','DistrictName']

schools_closures = schools_closures[columns_to_keep]

schools_closures['LEAID'] = schools_closures['NCESDistrictID'].apply(lambda x: '00' + x if len(x) == 5 else x)

schools_closures['NCESDistrictID'] = schools_closures['LEAID'].apply(lambda x: '0' + x if len(x) == 6 else x)

#for mid 2020 (taking two months in case the first is missing)

schools_closures_m2020 = schools_closures[schools_closures['month'] == "2020m8"].reset_index(drop=True)

schools_closures_m2020 = schools_closures_m2020.rename(columns={
    'share_inperson': 'august20_school_share_inperson'
})

schools_closures_m2020 = schools_closures_m2020[['LEAID','august20_school_share_inperson','DistrictName']]

#now mnth 9
schools_closures_m2020_2 = schools_closures[schools_closures['month'] == "2020m9"].reset_index(drop=True)

schools_closures_m2020_2 = schools_closures_m2020_2.rename(columns={
    'share_inperson': 'september20_school_share_inperson'
})

schools_closures_m2020_2 = schools_closures_m2020_2[['LEAID','september20_school_share_inperson']]

#for late 2020

schools_closures_l2020 = schools_closures[schools_closures['month'] == "2020m12"].reset_index(drop=True)

schools_closures_l2020 = schools_closures_l2020.rename(columns={
    'share_inperson': 'december20_school_share_inperson'
})

schools_closures_l2020 = schools_closures_l2020[['LEAID','december20_school_share_inperson']]

#now mnth 11

schools_closures_l2020_2 = schools_closures[schools_closures['month'] == "2020m11"].reset_index(drop=True)

schools_closures_l2020_2 = schools_closures_l2020_2.rename(columns={
    'share_inperson': 'november20_school_share_inperson'
})

schools_closures_l2020_2 = schools_closures_l2020_2[['LEAID','november20_school_share_inperson']]


#for mid 2021

schools_closures_m2021 = schools_closures[schools_closures['month'] == "2021m3"].reset_index(drop=True)

schools_closures_m2021 = schools_closures_m2021.rename(columns={
    'share_inperson': 'march21_school_share_inperson'
})

schools_closures_m2021_2 = schools_closures[schools_closures['month'] == "2021m2"].reset_index(drop=True)

schools_closures_m2021_2 = schools_closures_m2021_2.rename(columns={
    'share_inperson': 'feb21_school_share_inperson'
})


schools_closures_m2021 = schools_closures_m2021[['LEAID','march21_school_share_inperson']]

In [173]:
schools_closures = schools_closures[~schools_closures['LEAID'].duplicated()]

schools_closures = pd.merge(schools_closures, schools_closures_m2020, how = 'left', on = 'LEAID')
schools_closures = pd.merge(schools_closures, schools_closures_m2020_2, how = 'left', on = 'LEAID')
schools_closures = pd.merge(schools_closures, schools_closures_l2020_2, how = 'left', on = 'LEAID')
schools_closures = pd.merge(schools_closures, schools_closures_l2020, how = 'left', on = 'LEAID')
schools_closures = pd.merge(schools_closures, schools_closures_m2021_2, how = 'left', on = 'LEAID')
schools_closures = pd.merge(schools_closures, schools_closures_m2021, how = 'left', on = 'LEAID')

schools_closures['spring2020_school_closures'] = schools_closures.apply(
    lambda row: row['september20_school_share_inperson'] if pd.isna(row['august20_school_share_inperson']) else row['august20_school_share_inperson'], 
    axis=1
)

schools_closures['fall2020_school_closures'] = schools_closures['november20_school_share_inperson']

schools_closures['spring2021_school_closures'] = schools_closures.apply(
    lambda row: row['feb21_school_share_inperson'] if pd.isna(row['march21_school_share_inperson']) else row['march21_school_share_inperson'], 
    axis=1
)

schools_closures = schools_closures[['LEAID','spring2020_school_closures','fall2020_school_closures','spring2021_school_closures']]
schools_closures.to_csv('data/school_closure_merged.csv')

schools_closures

,LEAID,spring2020_school_closures,fall2020_school_closures,spring2021_school_closures
0,200001,.0032779209,.0029261156,.0032122559
1,200003,0,0,.03125
2,200004,0,0,0
3,200005,1,0,0
4,200006,1,1,1
...,...,...,...,...
14962,5605762,1,1,1
14963,5605820,1,1,1
14964,5605830,0,0,0
14965,5606090,1,1,1


In [174]:
ZIP_FEATURES['LEAID'] = ZIP_FEATURES['LEAID'].astype(str)

ZIP_FEATURES['LEAID'] = ZIP_FEATURES['LEAID'].apply(lambda x: '00' + x if len(x) == 5 else x)
ZIP_FEATURES['LEAID'] = ZIP_FEATURES['LEAID'].apply(lambda x: '0' + x if len(x) == 6 else x)

schools_closures['LEAID'] = schools_closures['LEAID'].apply(lambda x: '00' + x if len(x) == 5 else x)
schools_closures['LEAID'] = schools_closures['LEAID'].apply(lambda x: '0' + x if len(x) == 6 else x)

ZIP_FEATURES = pd.merge(ZIP_FEATURES, schools_closures, how = 'left', on = "LEAID")

ZIP_FEATURES

,ZIP,USPS_STATE,CD,COUNTY,TRACT,TRACT_LEAID,COUNTY_LEAID,TRACT_SCH_DISTRICT,COUNTY_SCH_DISTRICT,SCHOOL_DISTRICT,LEAID,spring2020_school_closures,fall2020_school_closures,spring2021_school_closures
0,01001,MA,2501,25013,25013813209,2501800,2505160,Agawam School District,Gateway School District,Agawam School District,2501800,NaN,0,0
1,01002,MA,2502,25015,25015820500,2501890,2505740,Amherst School District,Hampshire School District,Amherst School District,2501890,NaN,0,.90322578
2,01002,MA,2502,25015,25015820500,2501920,2505740,Amherst-Pelham School District,Hampshire School District,Amherst-Pelham School District,2501920,NaN,0,0
3,01004,MA,2502,25015,25015820500,2501890,2505740,Amherst School District,Hampshire School District,Amherst School District,2501890,NaN,0,.90322578
4,01004,MA,2502,25015,25015820500,2501920,2505740,Amherst-Pelham School District,Hampshire School District,Amherst-Pelham School District,2501920,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74408,99927,AK,0200,02198,02198000100,0200330,0200700,Hydaburg City School District,Southeast Island School District,Hydaburg City School District,0200330,NaN,NaN,NaN
74409,99927,AK,0200,02198,02198000100,0200360,0200700,Kake City School District,Southeast Island School District,Kake City School District,0200360,0,0,0
74410,99927,AK,0200,02198,02198000100,0200700,0200700,Southeast Island School District,Southeast Island School District,Southeast Island School District,0200700,.8897059,1,1
74411,99928,AK,0200,02130,02130000100,NaN,0200150,NaN,Ketchikan Gateway Borough School District,Ketchikan Gateway Borough School District,0200150,1,.55641276,1


Maybe it doesn't make sense to have ZIP to CD and ZIP to County in the same dataset. I might make these into two later on, but for now this contains the county under which the ZIP code mostly falls for residential addresses. 

In [55]:
ZIP_FEATURES['CD'] = ZIP_FEATURES['CD'].astype(str)
CD_df['CD'] = CD_df['CD'].astype(str)


ZIP_FEATURES = pd.merge(ZIP_FEATURES, CD_df, on = "CD", how= "left")
ZIP_FEATURES = ZIP_FEATURES.drop(columns=['FIPS_Code','Years_Served','District','CD_RES_RATIO',
                                                      'CD_BUS_RATIO','CD_OTH_RATIO','COUNTY_RES_RATIO',
                                                      'COUNTY_BUS_RATIO','COUNTY_OTH_RATIO'])

ZIP_FEATURES = ZIP_FEATURES.rename(columns={
    'COUNTY': 'COUNTY_FIPS',
    'Name': 'CONGRESSPERSON',
    'State': 'STATE',
    'Party': 'CONGRESSPERSON_PARTY'
})

cols_to_move = ZIP_FEATURES.columns[3:5]
df_remaining = ZIP_FEATURES.drop(columns=cols_to_move)
ZIP_FEATURES = pd.concat([df_remaining, ZIP_FEATURES[cols_to_move]], axis=1)

#ZIP_FEATURES['STATE'] = ZIP_FEATURES.apply(
    #lambda row: row['PHYSICAL STATE'] if pd.isna(row['STATE']) and not pd.isna(row['PHYSICAL STATE']) else row['STATE'], 
    #axis=1
#)

#ZIP_FEATURES = ZIP_FEATURES.drop(columns='PHYSICAL STATE')

ZIP_FEATURES

,ZIP,PHYSICAL STATE,CD,COUNTY_TOT_RATIO,NCESDistrictID,spring2020_school_closures,fall2020_school_closures,spring2021_school_closures,CONGRESSPERSON,STATE,CONGRESSPERSON_PARTY,CD_TOT_RATIO,COUNTY_FIPS
0,00601,PR,7200,0.837603,7200030,NaN,NaN,NaN,"González-Colón, Jenniffer",Puerto Rico,Republican,1.0,72001
1,00602,PR,7200,0.999919,7200030,NaN,NaN,NaN,"González-Colón, Jenniffer",Puerto Rico,Republican,1.0,72003
2,00603,PR,7200,0.997290,7200030,NaN,NaN,NaN,"González-Colón, Jenniffer",Puerto Rico,Republican,1.0,72005
3,00604,PR,7200,1.000000,7200030,NaN,NaN,NaN,"González-Colón, Jenniffer",Puerto Rico,Republican,1.0,72005
4,00605,PR,7200,1.000000,7200030,NaN,NaN,NaN,"González-Colón, Jenniffer",Puerto Rico,Republican,1.0,72005
...,...,...,...,...,...,...,...,...,...,...,...,...,...
59337,99811,AK,0200,1.000000,NaN,NaN,NaN,NaN,"Young, Don",Alaska,Republican,1.0,02110
59338,99812,AK,0200,NaN,NaN,NaN,NaN,NaN,"Young, Don",Alaska,Republican,1.0,NaN
59339,99850,AK,0200,NaN,NaN,NaN,NaN,NaN,"Young, Don",Alaska,Republican,1.0,NaN
59340,99903,AK,0200,1.000000,NaN,NaN,NaN,NaN,"Young, Don",Alaska,Republican,1.0,02275


Pulling from their API to see if I can extract any more years worth of data \
It looks like it's only cumulative.

In [56]:
import requests
import json

# API endpoint
url = "https://data.cdc.gov/resource/kn79-hsxy.json"

# Making a GET request
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    data = response.json()
    
    # Extracting all 'data_as_of' values
    data_as_of_values = [item['data_as_of'] for item in data if 'data_as_of' in item]

    # Getting unique values by converting the list to a set
    unique_data_as_of_values = set(data_as_of_values)

    # Printing unique 'data_as_of' values
    print("Unique 'data_as_of' values:")
    for value in unique_data_as_of_values:
        print(value)
else:
    print(f"Failed to retrieve data. Status code: {response.status_code}")

Unique 'data_as_of' values:
2023-06-28T00:00:00.000


Below, we need to add a leading "0" to the FIPS code (FIPS codes are all 5 digits) \
I'll borrw Urban Rural Code from here

In [57]:
col_types = {
    'FIPS County Code': str
}

CDC_deaths = pd.read_csv('Data/Partisanship and Health Behavior/Data/Provisional_COVID-19_Death_Counts_in_the_United_States_by_County.csv', dtype=col_types)
CDC_deaths['FIPS County Code'] = CDC_deaths['FIPS County Code'].astype(str).str.zfill(5)

CDC_deaths = CDC_deaths.drop(columns = ['Date as of','Deaths from All Causes','Footnote','State'])
CDC_deaths.head()

,Start Date,End Date,County name,FIPS County Code,Urban Rural Code,Deaths involving COVID-19
0,01/01/2020,06/24/2023,Aleutians East Borough,02013,Noncore,NaN
1,01/01/2020,06/24/2023,Anchorage Municipality,02020,Medium metro,775.0
2,01/01/2020,06/24/2023,Bethel Census Area,02050,Noncore,40.0
3,01/01/2020,06/24/2023,Denali Borough,02068,Noncore,NaN
4,01/01/2020,06/24/2023,Dillingham Census Area,02070,Noncore,NaN


In [58]:
col_types = {
    'countyFIPS': str
}

usafacts_deaths = pd.read_csv('Data/Partisanship and Health Behavior/Data/covid_deaths_usafacts.csv', dtype=col_types)
usafacts_deaths['countyFIPS'] = usafacts_deaths['countyFIPS'].astype(str).str.zfill(5)

For this preliminary analysis, I will only use wave 2,4,6. \
Each wave's data was collected on 2020-06-17 - 2020-06-23, 2020-11-29 - 2020-12-16, 2021-05-12 - 2021-05-25. \
I will take covid mortality from the midpoint of the wave range. \
Wave 2 = 6/18-24. 2020 \
Wave 3 = 9/12-20, 2020 \
Wave 4 = 12/1-7, 2020 \
Wave 5 = 2/9-14, 2021 \
Wave 6 = 5/14-25, 2021

In [59]:
columns_to_keep = ['countyFIPS', 'County Name', '2020-06-20', '2020-12-09', '2021-05-19']

usafacts_deaths = usafacts_deaths[columns_to_keep]

usafacts_deaths = usafacts_deaths.rename(columns={
    'countyFIPS': 'COUNTY_FIPS',
    '2020-06-20': 'COUNTY_COV_DEATHS_2020_06_20',
    '2020-12-09': 'COUNTY_COV_DEATHS_2020_12_09',
    '2021-05-19': 'COUNTY_COV_DEATHS_2021_05_19'
})

In [60]:
ZIP_FEATURES = pd.merge(ZIP_FEATURES, usafacts_deaths, on = "COUNTY_FIPS", how = "left")
ZIP_FEATURES

,ZIP,PHYSICAL STATE,CD,COUNTY_TOT_RATIO,NCESDistrictID,spring2020_school_closures,fall2020_school_closures,spring2021_school_closures,CONGRESSPERSON,STATE,CONGRESSPERSON_PARTY,CD_TOT_RATIO,COUNTY_FIPS,County Name,COUNTY_COV_DEATHS_2020_06_20,COUNTY_COV_DEATHS_2020_12_09,COUNTY_COV_DEATHS_2021_05_19
0,00601,PR,7200,0.837603,7200030,NaN,NaN,NaN,"González-Colón, Jenniffer",Puerto Rico,Republican,1.0,72001,NaN,NaN,NaN,NaN
1,00602,PR,7200,0.999919,7200030,NaN,NaN,NaN,"González-Colón, Jenniffer",Puerto Rico,Republican,1.0,72003,NaN,NaN,NaN,NaN
2,00603,PR,7200,0.997290,7200030,NaN,NaN,NaN,"González-Colón, Jenniffer",Puerto Rico,Republican,1.0,72005,NaN,NaN,NaN,NaN
3,00604,PR,7200,1.000000,7200030,NaN,NaN,NaN,"González-Colón, Jenniffer",Puerto Rico,Republican,1.0,72005,NaN,NaN,NaN,NaN
4,00605,PR,7200,1.000000,7200030,NaN,NaN,NaN,"González-Colón, Jenniffer",Puerto Rico,Republican,1.0,72005,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59337,99811,AK,0200,1.000000,NaN,NaN,NaN,NaN,"Young, Don",Alaska,Republican,1.0,02110,City and Borough of Juneau,0.0,3.0,3.0
59338,99812,AK,0200,NaN,NaN,NaN,NaN,NaN,"Young, Don",Alaska,Republican,1.0,NaN,NaN,NaN,NaN,NaN
59339,99850,AK,0200,NaN,NaN,NaN,NaN,NaN,"Young, Don",Alaska,Republican,1.0,NaN,NaN,NaN,NaN,NaN
59340,99903,AK,0200,1.000000,NaN,NaN,NaN,NaN,"Young, Don",Alaska,Republican,1.0,02275,City and Borough of Wrangell,0.0,0.0,0.0


In [61]:
col_types1 = {
    'STATE': str,
    'COUNTY': str
}

col_types2 = {
    'County Code (FIPS)': str,
    'County Subdivision Code (FIPS)': str,
    'Place Code (FIPS)': str,
    'Consolidtated City Code (FIPS)': str,
    'State Code (FIPS)': str
}


census_pop = pd.read_csv('Data/Partisanship and Health Behavior/Data/co-est2021-alldata.csv', encoding='ISO-8859-1',
                        dtype = col_types1)
census_pop_FIPS = pd.read_excel('Data/Partisanship and Health Behavior/Data/all-geocodes-v2020.xlsx', 
                                dtype = col_types2)

census_pop_FIPS = census_pop_FIPS.rename(columns={
    'Area Name (including legal/statistical area description)': 'CTYNAME'
})

census_pop['COUNTY_FIPS'] = census_pop['STATE']+census_pop['COUNTY']

State_FIPS_Crosswalk = census_pop[['STATE','STNAME']]
State_FIPS_Crosswalk['STATE_FIPS'] = State_FIPS_Crosswalk['STATE'].astype(str)
State_FIPS_Crosswalk['STATE'] = State_FIPS_Crosswalk['STNAME']

State_FIPS_Crosswalk = State_FIPS_Crosswalk.drop(columns = ['STNAME'])

State_FIPS_Crosswalk = State_FIPS_Crosswalk.drop_duplicates(subset='STATE', keep='first')
State_FIPS_Crosswalk = State_FIPS_Crosswalk[pd.notna(State_FIPS_Crosswalk['STATE'])].reset_index(drop=True)

State_FIPS_Crosswalk.to_csv('data/STATE_FIPS_CROSSWALK.csv', index = False)
State_FIPS_Crosswalk.to_csv('data/Partisanship and Health Behavior/Data/STATE_FIPS_CROSSWALK.csv', index = False)

columns_to_keep = ['STATE','POPESTIMATE2020','DEATHS2020','POPESTIMATE2021','DEATHS2021','COUNTY_FIPS']

census_pop = census_pop[columns_to_keep]

census_pop = census_pop.rename(columns={
    'STATE': 'STATE_FIPS'
})

census_pop

/var/folders/89/6bdxzk2j30v5n3wstywbcpg80000gn/T/ipykernel_63857/2112413992.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  State_FIPS_Crosswalk['STATE_FIPS'] = State_FIPS_Crosswalk['STATE'].astype(str)
/var/folders/89/6bdxzk2j30v5n3wstywbcpg80000gn/T/ipykernel_63857/2112413992.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  State_FIPS_Crosswalk['STATE'] = State_FIPS_Crosswalk['STNAME']


,STATE_FIPS,POPESTIMATE2020,DEATHS2020,POPESTIMATE2021,DEATHS2021,COUNTY_FIPS
0,01,5024803,16148,5039877,64868,01000
1,01,58877,168,59095,681,01001
2,01,233140,661,239294,2867,01003
3,01,25180,109,24964,394,01005
4,01,22223,90,22477,282,01007
...,...,...,...,...,...,...
3189,56,42158,116,41614,400,56037
3190,56,23347,29,23575,96,56039
3191,56,20441,49,20635,166,56041
3192,56,7658,32,7705,115,56043


A crude mortality rate, not standardized or age adjusted, for this preliminary analysis \
I manually insert Puerto Rico' State FIPS code and District of Columbia to make this a clean merge across the board.

In [62]:
ZIP_FEATURES = pd.merge(ZIP_FEATURES, census_pop, on = "COUNTY_FIPS", how = "left")


ZIP_FEATURES['COV_County_CMR_2020_06_20'] = (ZIP_FEATURES['COUNTY_COV_DEATHS_2020_06_20'] / ZIP_FEATURES['POPESTIMATE2020']) * 1000
ZIP_FEATURES['COV_County_CMR_2020_12_09'] = (ZIP_FEATURES['COUNTY_COV_DEATHS_2020_12_09'] / ZIP_FEATURES['POPESTIMATE2020']) * 1000
ZIP_FEATURES['COV_County_CMR_2021_05_19'] = (ZIP_FEATURES['COUNTY_COV_DEATHS_2021_05_19'] / ZIP_FEATURES['POPESTIMATE2021']) * 1000
ZIP_FEATURES['STATE_FIPS'] = ZIP_FEATURES.apply(lambda row: 72 if row['STATE'] == "Puerto Rico" else row['STATE_FIPS'], axis=1)

ZIP_FEATURES

,ZIP,PHYSICAL STATE,CD,COUNTY_TOT_RATIO,NCESDistrictID,spring2020_school_closures,fall2020_school_closures,spring2021_school_closures,CONGRESSPERSON,STATE,...,COUNTY_COV_DEATHS_2020_12_09,COUNTY_COV_DEATHS_2021_05_19,STATE_FIPS,POPESTIMATE2020,DEATHS2020,POPESTIMATE2021,DEATHS2021,COV_County_CMR_2020_06_20,COV_County_CMR_2020_12_09,COV_County_CMR_2021_05_19
0,00601,PR,7200,0.837603,7200030,NaN,NaN,NaN,"González-Colón, Jenniffer",Puerto Rico,...,NaN,NaN,72,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00602,PR,7200,0.999919,7200030,NaN,NaN,NaN,"González-Colón, Jenniffer",Puerto Rico,...,NaN,NaN,72,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,00603,PR,7200,0.997290,7200030,NaN,NaN,NaN,"González-Colón, Jenniffer",Puerto Rico,...,NaN,NaN,72,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00604,PR,7200,1.000000,7200030,NaN,NaN,NaN,"González-Colón, Jenniffer",Puerto Rico,...,NaN,NaN,72,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00605,PR,7200,1.000000,7200030,NaN,NaN,NaN,"González-Colón, Jenniffer",Puerto Rico,...,NaN,NaN,72,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59337,99811,AK,0200,1.000000,NaN,NaN,NaN,NaN,"Young, Don",Alaska,...,3.0,3.0,02,32195.0,52.0,31973.0,204.0,0.0,0.093182,0.093829
59338,99812,AK,0200,NaN,NaN,NaN,NaN,NaN,"Young, Don",Alaska,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59339,99850,AK,0200,NaN,NaN,NaN,NaN,NaN,"Young, Don",Alaska,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59340,99903,AK,0200,1.000000,NaN,NaN,NaN,NaN,"Young, Don",Alaska,...,0.0,0.0,02,2130.0,5.0,2055.0,67.0,0.0,0.000000,0.000000


It might be more useful to have data on prevalence \
I want these lags \
Wave 2 (6/18-24, 2020), midpoint: 06/21/2020, lag1: 06/14/2020, lag3: 06/07/2020 \
Wave 3 (9/12-20, 2020), midpoint: 09/16/2020, lag1: 09/09/2020, lag3: 09/02/2020 \
Wave 4 (12/1-7, 2020), midpoint: 12/04/2020, lag1: 11/27/2020, lag3: 11/20/2020 \
Wave 5 (2/9-14, 2021), midpoint: 02/11/2021, lag: 02/04/2021, lag3: 01/28/2021 \
Wave 6 (5/14-25, 2021), midpoint: 05/19/2021, lag: 05/12/2021, lag3: 05/05/2021 

In [63]:
col_types = {
    'FIPS': str,
    'iso2':str
}


prevalence_mid_2020 = pd.read_csv('Data/Partisanship and Health Behavior/Data/JH_Prevalence/time_series_covid19_confirmed_US_mid_2020.csv', 
                                  dtype = col_types)

columns_to_keep = ['FIPS','6/21/20','6/18/20','6/14/20','6/7/20']

prevalence_mid_2020 = prevalence_mid_2020[prevalence_mid_2020['iso2'] == 'US']
prevalence_mid_2020['FIPS'] = prevalence_mid_2020['FIPS'].apply(lambda x: str(x).rstrip('0').rstrip('.') if pd.notna(x) and '.' in str(x) else x)
prevalence_mid_2020['FIPS'] = prevalence_mid_2020['FIPS'].apply(lambda x: str(x).zfill(5))

prevalence_mid_2020 = prevalence_mid_2020[columns_to_keep]

prevalence_late_2020 =pd.read_csv('Data/Partisanship and Health Behavior/Data/JH_Prevalence/time_series_covid19_confirmed_US_late_2020.csv', 
                                  dtype = col_types)

columns_to_keep = ['FIPS','9/16/20','9/12/20','9/9/20','9/2/20']

prevalence_late_2020 = prevalence_late_2020[prevalence_late_2020['iso2'] == 'US']
prevalence_late_2020['FIPS'] = prevalence_late_2020['FIPS'].apply(lambda x: str(x).rstrip('0').rstrip('.') if pd.notna(x) and '.' in str(x) else x)
prevalence_late_2020['FIPS'] = prevalence_late_2020['FIPS'].apply(lambda x: str(x).zfill(5))

prevalence_late_2020 = prevalence_late_2020[columns_to_keep]

prevalence_early_2021 = pd.read_csv('Data/Partisanship and Health Behavior/Data/JH_Prevalence/time_series_covid19_confirmed_US_early_2021.csv', 
                                  dtype = col_types)

columns_to_keep = ['FIPS','12/4/20','12/1/20','11/27/20','11/20/20']

prevalence_early_2021 = prevalence_early_2021[prevalence_early_2021['iso2'] == 'US']
prevalence_early_2021['FIPS'] = prevalence_early_2021['FIPS'].apply(lambda x: str(x).rstrip('0').rstrip('.') if pd.notna(x) and '.' in str(x) else x)
prevalence_early_2021['FIPS'] = prevalence_early_2021['FIPS'].apply(lambda x: str(x).zfill(5))

prevalence_early_2021 = prevalence_early_2021[columns_to_keep]

prevalence_mid_2021 = pd.read_csv('Data/Partisanship and Health Behavior/Data/JH_Prevalence/time_series_covid19_confirmed_US_mid_2021.csv', 
                                  dtype = col_types)

columns_to_keep = ['FIPS','2/11/21','2/9/21','2/4/21','1/28/21','5/19/21','5/14/21','5/12/21','5/5/21']

prevalence_mid_2021 = prevalence_mid_2021[prevalence_mid_2021['iso2'] == 'US']
prevalence_mid_2021['FIPS'] = prevalence_mid_2021['FIPS'].apply(lambda x: str(x).rstrip('0').rstrip('.') if pd.notna(x) and '.' in str(x) else x)
prevalence_mid_2021['FIPS'] = prevalence_mid_2021['FIPS'].apply(lambda x: str(x).zfill(5))

prevalence_mid_2021 = prevalence_mid_2021[columns_to_keep]

prevalence = pd.merge(prevalence_mid_2020, prevalence_late_2020, how = "left", on = "FIPS")
prevalence = pd.merge(prevalence, prevalence_early_2021, how = "left", on = "FIPS")
prevalence = pd.merge(prevalence, prevalence_mid_2021, how = "left", on = "FIPS")
prevalence.columns = [col if i == 0 else 'prev_' + col for i, col in enumerate(prevalence.columns)]

prevalence = prevalence.rename(columns={
    'FIPS':'COUNTY_FIPS'
})

prevalence.head()

,COUNTY_FIPS,prev_6/21/20,prev_6/18/20,prev_6/14/20,prev_6/7/20,prev_9/16/20,prev_9/12/20,prev_9/9/20,prev_9/2/20,prev_12/4/20,...,prev_11/27/20,prev_11/20/20,prev_2/11/21,prev_2/9/21,prev_2/4/21,prev_1/28/21,prev_5/19/21,prev_5/14/21,prev_5/12/21,prev_5/5/21
0,01001,428,394,357,265,1619,1432,1398,1345,2945,...,2716,2580,5970,5910,5811,5554,7017,6971,6928,6914
1,01003,415,386,364,313,5003,4752,4639,4458,9501,...,8603,8038,18960,18824,18494,17779,21467,21290,21170,21035
2,01005,271,262,234,193,809,620,618,617,1217,...,1171,1151,2030,2019,1989,1920,2324,2319,2314,2307
3,01007,124,118,104,77,612,576,564,541,1270,...,1173,1024,2377,2362,2327,2271,2652,2630,2612,2604
4,01009,146,127,110,72,1487,1114,1082,1036,3231,...,2888,2704,5955,5929,5842,5612,6808,6750,6680,6635


In [64]:
ZIP_FEATURES = pd.merge(ZIP_FEATURES, prevalence, on = "COUNTY_FIPS", how = "left")

for col in ZIP_FEATURES.columns:
    # Check if the column name starts with 'prev_'
    if col.startswith('prev_'):
        # Extract the date part of the column name
        date_part = col.split('_', 1)[1]
        # Perform the calculation and create a new column
        ZIP_FEATURES['rate_prev_' + date_part] = (ZIP_FEATURES[col] / ZIP_FEATURES['POPESTIMATE2020']) * 1000
        
prev_columns = prevalence.columns
prev_columns = prev_columns[1:]
ZIP_FEATURES = ZIP_FEATURES.drop(columns=prev_columns)

Below I insert data on urbanicity

In [65]:
columns_to_keep = ['FIPS','RUCC_2013']

col_types = {
    'FIPS': str,
}

RUC_county_df = pd.read_csv('Data/Partisanship and Health Behavior/Data/ruralurbancodes2013.csv', dtype = col_types)

RUC_county_df = RUC_county_df[columns_to_keep]

RUC_county_df = RUC_county_df.rename(columns={
    'RUCC_2013': 'COUNTY_RUCC_2013',
    'FIPS':'COUNTY_FIPS'
})

RUC_county_df['COUNTY_FIPS'] = RUC_county_df['COUNTY_FIPS'].astype(str)

ZIP_FEATURES = pd.merge(ZIP_FEATURES, RUC_county_df, on = "COUNTY_FIPS", how = "left")

Next, I want to control for the severity of the lockdowns according to Oxford's tracker

In [66]:
State_Restrictions_df = pd.read_csv('Data/Partisanship and Health Behavior/Data/OxCGRT_USA_differentiated_withnotes_2020.csv')
State_Restrictions_df['Date'] = State_Restrictions_df['Date'].astype(str)
State_Restrictions_df = State_Restrictions_df[State_Restrictions_df['Date'].isin(['20200617', '20201129'])]
columns_to_keep = ['RegionName','GovernmentResponseIndex_SimpleAverage','Date']
State_Restrictions_df = State_Restrictions_df[columns_to_keep]

State_Restrictions_df = State_Restrictions_df.rename(columns={
    'RegionName': 'STATE',
    'GovernmentResponseIndex_SimpleAverage':'State_Government_Response_Index'
})

State_Restrictions_df

/var/folders/89/6bdxzk2j30v5n3wstywbcpg80000gn/T/ipykernel_63857/750958945.py:1: DtypeWarning: Columns (82,93,121,122) have mixed types. Specify dtype option on import or set low_memory=False.
  State_Restrictions_df = pd.read_csv('Data/Partisanship and Health Behavior/Data/OxCGRT_USA_differentiated_withnotes_2020.csv')


,STATE,State_Government_Response_Index,Date
168,NaN,62.50,20200617
333,NaN,67.19,20201129
534,Alaska,57.81,20200617
699,Alaska,53.65,20201129
900,Alabama,59.64,20200617
...,...,...,...
18267,Wisconsin,53.65,20201129
18468,West Virginia,61.46,20200617
18633,West Virginia,56.25,20201129
18834,Wyoming,59.90,20200617


In [67]:
State_Restrictions_df['State_Government_Response_Index_06172020'] = State_Restrictions_df.apply(
    lambda row: row['State_Government_Response_Index'] if row['Date'] == "20200617" else "0", axis=1
)

State_Restrictions_df['State_Government_Response_Index_06172020'] = State_Restrictions_df.apply(
    lambda row: row['State_Government_Response_Index'] if row['State_Government_Response_Index_06172020'] == "0" else row['State_Government_Response_Index_06172020'], axis=1
)

State_Restrictions_df['State_Government_Response_Index_11192020'] = State_Restrictions_df.apply(
    lambda row: row['State_Government_Response_Index'] if row['Date'] == "20201129" else "0", axis=1
)

State_Restrictions_df['State_Government_Response_Index_11192020'] = State_Restrictions_df.apply(
    lambda row: row['State_Government_Response_Index'] if row['State_Government_Response_Index_11192020'] == "0" else row['State_Government_Response_Index_11192020'], axis=1
)

State_Restrictions_df

,STATE,State_Government_Response_Index,Date,State_Government_Response_Index_06172020,State_Government_Response_Index_11192020
168,NaN,62.50,20200617,62.50,62.50
333,NaN,67.19,20201129,67.19,67.19
534,Alaska,57.81,20200617,57.81,57.81
699,Alaska,53.65,20201129,53.65,53.65
900,Alabama,59.64,20200617,59.64,59.64
...,...,...,...,...,...
18267,Wisconsin,53.65,20201129,53.65,53.65
18468,West Virginia,61.46,20200617,61.46,61.46
18633,West Virginia,56.25,20201129,56.25,56.25
18834,Wyoming,59.90,20200617,59.90,59.90


In [68]:
ZIP_FEATURES = pd.merge(ZIP_FEATURES, State_Restrictions_df, on = "STATE", how = "left")

ZIP_FEATURES

,ZIP,PHYSICAL STATE,CD,COUNTY_TOT_RATIO,NCESDistrictID,spring2020_school_closures,fall2020_school_closures,spring2021_school_closures,CONGRESSPERSON,STATE,...,rate_prev_1/28/21,rate_prev_5/19/21,rate_prev_5/14/21,rate_prev_5/12/21,rate_prev_5/5/21,COUNTY_RUCC_2013,State_Government_Response_Index,Date,State_Government_Response_Index_06172020,State_Government_Response_Index_11192020
0,00601,PR,7200,0.837603,7200030,NaN,NaN,NaN,"González-Colón, Jenniffer",Puerto Rico,...,NaN,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN
1,00602,PR,7200,0.999919,7200030,NaN,NaN,NaN,"González-Colón, Jenniffer",Puerto Rico,...,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN
2,00603,PR,7200,0.997290,7200030,NaN,NaN,NaN,"González-Colón, Jenniffer",Puerto Rico,...,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN
3,00604,PR,7200,1.000000,7200030,NaN,NaN,NaN,"González-Colón, Jenniffer",Puerto Rico,...,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN
4,00605,PR,7200,1.000000,7200030,NaN,NaN,NaN,"González-Colón, Jenniffer",Puerto Rico,...,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118490,99850,AK,0200,NaN,NaN,NaN,NaN,NaN,"Young, Don",Alaska,...,NaN,NaN,NaN,NaN,NaN,NaN,53.65,20201129,53.65,53.65
118491,99903,AK,0200,1.000000,NaN,NaN,NaN,NaN,"Young, Don",Alaska,...,12.206573,26.760563,26.760563,26.760563,26.29108,9.0,57.81,20200617,57.81,57.81
118492,99903,AK,0200,1.000000,NaN,NaN,NaN,NaN,"Young, Don",Alaska,...,12.206573,26.760563,26.760563,26.760563,26.29108,9.0,53.65,20201129,53.65,53.65
118493,99950,AK,0200,NaN,NaN,NaN,NaN,NaN,"Young, Don",Alaska,...,NaN,NaN,NaN,NaN,NaN,NaN,57.81,20200617,57.81,57.81


Next, I want to read in the county level presidential data and generate two variables \
1. For which president did the county overall vote for?
2. What percentage of the county voted for which president?

note: I manually added in DC because it was not in the dataset

In [69]:
col_types = {
    'county_fips': str,
}
county_pres = pd.read_csv('Data/Partisanship and Health Behavior/Data/countypres_2000-2020.csv', dtype = col_types)
county_pres = county_pres[county_pres['year'] == 2020].reset_index(drop=True)

county_pres['party_fips'] = county_pres['party'].astype(str) + county_pres['county_fips'].astype(str)

#there was an issue where there were multiple types of votes on each line for the same person
total_votes_by_party = county_pres.groupby('party_fips')['candidatevotes'].sum().reset_index()
total_votes_by_party.rename(columns={'candidatevotes': 'total_candidatevotes'}, inplace=True)

county_pres = county_pres.merge(total_votes_by_party, on='party_fips', how='left')

county_pres['percentage_won'] = (county_pres['total_candidatevotes'] / county_pres['totalvotes'])*100

# Group by 'countyfips' and find the index of the row with the largest 'percentage_won' in each group
county_pres_wide = county_pres.groupby('county_fips')['percentage_won'].idxmax()

# Select rows from the original DataFrame corresponding to these indices
county_pres_wide = county_pres.loc[county_pres_wide]

county_pres_wide = county_pres_wide.rename(columns={
    'candidate': 'winning_pres_candidate',
    'party':'winning_pres_party',
    'percentage_won': 'winning_pres_candidate_percentage'
})

#merge in the dems
dem_county_pres = county_pres[county_pres['party'] == 'DEMOCRAT']

dem_county_pres.rename(columns={'percentage_won': 'biden_percentage_won'}, inplace=True)

dem_county_pres = dem_county_pres[['biden_percentage_won','county_fips']]

county_pres_wide = county_pres_wide.merge(dem_county_pres, on='county_fips', how='left')

#merge in the reps
rep_county_pres = county_pres[county_pres['party'] == 'REPUBLICAN']

rep_county_pres.rename(columns={'percentage_won': 'trump_percentage_won'}, inplace=True)

rep_county_pres = rep_county_pres[['trump_percentage_won','county_fips']]

county_pres_wide = county_pres_wide.merge(rep_county_pres, on='county_fips', how='left')

#keeping only the columns I want
county_pres_wide = county_pres_wide.drop(columns =['year','state','state_po','county_name','office','candidatevotes','totalvotes','version','mode','party_fips'])

county_pres_wide

/var/folders/89/6bdxzk2j30v5n3wstywbcpg80000gn/T/ipykernel_63857/345468839.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dem_county_pres.rename(columns={'percentage_won': 'biden_percentage_won'}, inplace=True)
/var/folders/89/6bdxzk2j30v5n3wstywbcpg80000gn/T/ipykernel_63857/345468839.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rep_county_pres.rename(columns={'percentage_won': 'trump_percentage_won'}, inplace=True)


,county_fips,winning_pres_candidate,winning_pres_party,total_candidatevotes,winning_pres_candidate_percentage,biden_percentage_won,trump_percentage_won
0,01001,DONALD J TRUMP,REPUBLICAN,19838,71.436802,27.018365,71.436802
1,01003,DONALD J TRUMP,REPUBLICAN,83544,76.171373,22.409030,76.171373
2,01005,DONALD J TRUMP,REPUBLICAN,5622,53.451226,45.788173,53.451226
3,01007,DONALD J TRUMP,REPUBLICAN,7525,78.426264,20.698280,78.426264
4,01009,DONALD J TRUMP,REPUBLICAN,24711,89.571553,9.569378,89.571553
...,...,...,...,...,...,...,...
12865,56037,DONALD J TRUMP,REPUBLICAN,12229,73.236316,22.894957,73.236316
12866,56039,JOSEPH R BIDEN JR,DEMOCRAT,9848,66.599040,66.599040,29.356868
12867,56041,DONALD J TRUMP,REPUBLICAN,7496,79.247278,16.819960,79.247278
12868,56043,DONALD J TRUMP,REPUBLICAN,3245,80.481151,16.145833,80.481151


In [70]:
county_pres_wide = county_pres_wide.rename(columns={
    'county_fips': 'COUNTY_FIPS'
})

ZIP_FEATURES = pd.merge(ZIP_FEATURES, county_pres_wide, on = "COUNTY_FIPS", how = "left")

ZIP_FEATURES

,ZIP,PHYSICAL STATE,CD,COUNTY_TOT_RATIO,NCESDistrictID,spring2020_school_closures,fall2020_school_closures,spring2021_school_closures,CONGRESSPERSON,STATE,...,State_Government_Response_Index,Date,State_Government_Response_Index_06172020,State_Government_Response_Index_11192020,winning_pres_candidate,winning_pres_party,total_candidatevotes,winning_pres_candidate_percentage,biden_percentage_won,trump_percentage_won
0,00601,PR,7200,0.837603,7200030,NaN,NaN,NaN,"González-Colón, Jenniffer",Puerto Rico,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00602,PR,7200,0.999919,7200030,NaN,NaN,NaN,"González-Colón, Jenniffer",Puerto Rico,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,00603,PR,7200,0.997290,7200030,NaN,NaN,NaN,"González-Colón, Jenniffer",Puerto Rico,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00604,PR,7200,1.000000,7200030,NaN,NaN,NaN,"González-Colón, Jenniffer",Puerto Rico,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00605,PR,7200,1.000000,7200030,NaN,NaN,NaN,"González-Colón, Jenniffer",Puerto Rico,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
410712,99850,AK,0200,NaN,NaN,NaN,NaN,NaN,"Young, Don",Alaska,...,53.65,20201129,53.65,53.65,NaN,NaN,NaN,NaN,NaN,NaN
410713,99903,AK,0200,1.000000,NaN,NaN,NaN,NaN,"Young, Don",Alaska,...,57.81,20200617,57.81,57.81,NaN,NaN,NaN,NaN,NaN,NaN
410714,99903,AK,0200,1.000000,NaN,NaN,NaN,NaN,"Young, Don",Alaska,...,53.65,20201129,53.65,53.65,NaN,NaN,NaN,NaN,NaN,NaN
410715,99950,AK,0200,NaN,NaN,NaN,NaN,NaN,"Young, Don",Alaska,...,57.81,20200617,57.81,57.81,NaN,NaN,NaN,NaN,NaN,NaN


Next, for the sake of consistency, I want to get data on the percentage of the CD that was won by a democrat and Republican, respectively. 

However, I need a clean crosswalk (as this one is pulling out several duplicates)

In [71]:
col_types = {
    'DISTRICT': str,
}

county_CD_share = pd.read_excel('Data/Partisanship and Health Behavior/Data/federalelections2020.xlsx', 
                                sheet_name='13_US_House_Results_State', 
                                dtype=col_types)

county_CD_share = county_CD_share.rename(columns={
    'DISTRICT': 'CD_DISTRICT'
})

county_CD_share = county_CD_share[pd.notna(county_CD_share['GENERAL %'])].reset_index(drop=True)

county_CD_share = pd.merge(county_CD_share, State_FIPS_Crosswalk, how='left', on="STATE")

county_CD_share['CD'] = county_CD_share['STATE_FIPS'] + county_CD_share['CD_DISTRICT']

county_CD_share.to_csv('data/CD_COUNTY_PERCENT_WON.csv')

D_county_CD_share = county_CD_share[county_CD_share['PARTY'] == "D"].reset_index(drop=True)
D_county_CD_share = D_county_CD_share[['GENERAL %','CD']]

D_county_CD_share = D_county_CD_share.rename(columns={
    'GENERAL %': 'CD_PERCENT_DEMOCRAT'
})

R_county_CD_share = county_CD_share[county_CD_share['PARTY'] == "R"].reset_index(drop=True)
R_county_CD_share = R_county_CD_share[['GENERAL %','CD']]

R_county_CD_share = R_county_CD_share.rename(columns={
    'GENERAL %': 'CD_PERCENT_REPUBLICAN'
})

ZIP_FEATURES = pd.merge(ZIP_FEATURES, D_county_CD_share, how = 'left', on = "CD")
ZIP_FEATURES = pd.merge(ZIP_FEATURES, R_county_CD_share, how = 'left', on = "CD")

ZIP_FEATURES

,ZIP,PHYSICAL STATE,CD,COUNTY_TOT_RATIO,NCESDistrictID,spring2020_school_closures,fall2020_school_closures,spring2021_school_closures,CONGRESSPERSON,STATE,...,State_Government_Response_Index_06172020,State_Government_Response_Index_11192020,winning_pres_candidate,winning_pres_party,total_candidatevotes,winning_pres_candidate_percentage,biden_percentage_won,trump_percentage_won,CD_PERCENT_DEMOCRAT,CD_PERCENT_REPUBLICAN
0,00601,PR,7200,0.837603,7200030,NaN,NaN,NaN,"González-Colón, Jenniffer",Puerto Rico,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00602,PR,7200,0.999919,7200030,NaN,NaN,NaN,"González-Colón, Jenniffer",Puerto Rico,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,00603,PR,7200,0.997290,7200030,NaN,NaN,NaN,"González-Colón, Jenniffer",Puerto Rico,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00604,PR,7200,1.000000,7200030,NaN,NaN,NaN,"González-Colón, Jenniffer",Puerto Rico,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00605,PR,7200,1.000000,7200030,NaN,NaN,NaN,"González-Colón, Jenniffer",Puerto Rico,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421172,99850,AK,0200,NaN,NaN,NaN,NaN,NaN,"Young, Don",Alaska,...,53.65,53.65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.544012
421173,99903,AK,0200,1.000000,NaN,NaN,NaN,NaN,"Young, Don",Alaska,...,57.81,57.81,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.544012
421174,99903,AK,0200,1.000000,NaN,NaN,NaN,NaN,"Young, Don",Alaska,...,53.65,53.65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.544012
421175,99950,AK,0200,NaN,NaN,NaN,NaN,NaN,"Young, Don",Alaska,...,57.81,57.81,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.544012


In [72]:
ZIP_FEATURES.to_csv('data/ZIP_Features.csv', index = False)

In [73]:
ZIP_FEATURES.to_csv('data/Partisanship and Health Behavior/Data/ZIP_Features.csv', index = False)

Why is resp_educ and education different?